# Connecting to a port in the container

Another common use case is that we start some kind of long-running process in the container, and talk to it through a port. That process could be a Jupyter Notebook, for example. 

## Example http.server

For demonstration purposes, we will use Python's in-built web server. To run it from the host, we could use

In [1]:
# NBVAL_SKIP
!python -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [08/Sep/2018 21:41:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2018 21:41:27] code 404, message File not found
127.0.0.1 - - [08/Sep/2018 21:41:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2018 21:41:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2018 21:41:31] code 404, message File not found
127.0.0.1 - - [08/Sep/2018 21:41:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2018 21:41:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2018 21:41:34] code 404, message File not found
127.0.0.1 - - [08/Sep/2018 21:41:34] "GET /favicon.ico HTTP/1.1" 404 -
^C

Keyboard interrupt received, exiting.


This program will show the contents of the current file system in a webbrowser interface at port 8000 of this machine. So typically at one or some of these links http://127.0.0.1:8000 or http://localhost:8000 or http://0.0.0.0:8000).

(If you have executed the above cell by pressing SHIFT+RETURN, you need to interrupt the http.server process to get the control back in the notebook. This can be done by choosing from the menu "Kernel" -> "Interrupt".)

We will now create a container and run this server inside the container. We like to use a webbrowser on the host machine to inspect the files.

First, we create the Dockerfile:

In [2]:
%%file Dockerfile
FROM ubuntu:18.04

RUN apt-get -y update
RUN apt-get -y install python3

CMD python3 -m http.server

Overwriting Dockerfile


The last line starts the `http.server` when the container is run.

In [3]:
#NBVAL_IGNORE_OUTPUT
!docker build -t portdemo .

Sending build context to Docker daemon  163.3kB
Step 1/4 : FROM ubuntu:18.04
 ---> cd6d8154f1e1
Step 2/4 : RUN apt-get -y update
 ---> Running in bd009d15a49b
Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe Sources [11.5 MB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/universe Sources [17.4 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [1363 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [69.0 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [203 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/

Selecting previously unselected package libmagic1:amd64.
Preparing to unpack .../libmagic1_1%3a5.32-2ubuntu0.1_amd64.deb ...
Unpacking libmagic1:amd64 (1:5.32-2ubuntu0.1) ...
Selecting previously unselected package file.
Preparing to unpack .../file_1%3a5.32-2ubuntu0.1_amd64.deb ...
Unpacking file (1:5.32-2ubuntu0.1) ...
Selecting previously unselected package xz-utils.
Preparing to unpack .../xz-utils_5.2.2-1.3_amd64.deb ...
Unpacking xz-utils (5.2.2-1.3) ...
Setting up readline-common (7.0-3) ...
Setting up mime-support (3.60ubuntu1) ...
Setting up libreadline7:amd64 (7.0-3) ...
Setting up libmagic-mgc (1:5.32-2ubuntu0.1) ...
Setting up libmagic1:amd64 (1:5.32-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up xz-utils (5.2.2-1.3) ...
update-alternatives: using /usr/bin/xz to provide /usr/bin/lzma (lzma) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/lzma.1.gz because associated file /usr/share/man/man1/xz.1.gz (of link gr

We now need to export the port 8000 in the container. We can do this using:

In [4]:
#NBVAL_SKIP
!docker run -p 8123:8000 portdemo

^C


The numbers `8123:8000` mean that the internal port (8000) of the container should be connected to the port (8123) on the host system.

Once the above command is executing, we should be able to browse the file system in the container by going to the link http://localhost:8123 (or http://127.0.0.1:8123 or http://0.0.0.0:8123) on the host system. 

We could have mapped port 8000 in the container to port 8000 on the host as well (`-p 8000:8000`).

As before, to stop the process, select `Kernel->Interrupt`.

## Jupyter Notebook

A common application of exposing ports is to install computational or data analysis software inside the container, and to control it from a Jupyter notebook running inside the container, but to use a webbrowser from the host system to interact with the notebook. In that case, the above example of exposing the port is in principle the right way to go, too. However, as this is a common usecase, there are a number of prepared Dockerfiles to install the notebook inside the container available at https://github.com/jupyter/docker-stacks, so that one can start the Dockerfile with `FROM jupyter/...`, (instead of `FROM ubuntu/...`) and in this way build on the Dockerfiles that the Jupyter team provides already.

The container image for https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook might be a good starting point for work based on the Scientific Python stack.